In [1]:
%load_ext autoreload

import threading
import time
import numpy as np

**First Demonstrate the Race Condition**

In [2]:
x = 0

def increment():
    global x
    x += 1
 
def thread1_task():
    for _ in range(100000):
        increment()
 
def thread2_task():
    for _ in range(100000):
        increment()
 
def main_task():
    global x
    x = 0
   
    t1 = threading.Thread(target=thread1_task)
    t2 = threading.Thread(target=thread2_task)
 
    t1.start()
    t2.start()
 
    t1.join()
    t2.join()

for i in range(10):
    main_task()
    print("Iteration {0}: x = {1}".format(i,x))

Iteration 0: x = 200000
Iteration 1: x = 167174
Iteration 2: x = 200000
Iteration 3: x = 166438
Iteration 4: x = 200000
Iteration 5: x = 200000
Iteration 6: x = 200000
Iteration 7: x = 200000
Iteration 8: x = 178791
Iteration 9: x = 200000


**Implement the 1st "Solution"**

In [7]:
class Solution_1:

    def __init__(self):
        self.turn = 0

    def lock(self, thread_ID):
        while self.turn != thread_ID: pass # spin

    def unlock(self, thread_ID):
        if thread_ID == 2: self.turn = 1
        else: self.turn = 2

x = 0

lock = Solution_1()


 
def increment():
    global x
    x += 1
 
def thread1_task(lock, my_num):
    global turn
   
    for _ in range(1000):

        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def thread2_task(lock, my_num):
    global turn
   
    for _ in range(1000):

        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def main_task():
    global x
    global lock
 
    x = 0
   
    t1 = threading.Thread(target=thread1_task, args=(lock, 1, ))
    t2 = threading.Thread(target=thread2_task, args=(lock, 2, ))
 
    t1.start()
    t2.start()
 
    t1.join()
    t2.join()

**Test the First "Solution"**

In [8]:
for i in range(10):
    main_task()
    print("Iteration {0}: x = {1}".format(i,x))

Iteration 0: x = 2000
Iteration 1: x = 2000
Iteration 2: x = 2000
Iteration 3: x = 2000
Iteration 4: x = 2000
Iteration 5: x = 2000
Iteration 6: x = 2000
Iteration 7: x = 2000
Iteration 8: x = 2000
Iteration 9: x = 2000


Mutual exclusion acheived! with only *minor* performance consequences. I had to reduce iterations to 1000 (2 orders of magnitude) to get it to run in reasonable time. 

In [3]:
flag = [False, False]

class Solution_2:

    def lock(self, thread_ID):
        global flag
        
        if thread_ID == 0:
            flag[0] = True
            while flag[1]: pass
        else:
            flag[1] = True
            while flag[0]: pass

    def unlock(self, thread_ID):
        global flag

        if thread_ID == 0:
            flag[0] = False
        else:
            flag[1] = False

def increment():
    global x
    x += 1
 
def thread1_task(lock, my_num):
    global turn
   
    for _ in range(10000):

        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def thread2_task(lock, my_num):
    global turn
   
    for _ in range(10000):

        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def main_task():
    global x
 
    x = 0

    lock = Solution_2()
   
    t1 = threading.Thread(target=thread1_task, args=(lock, 0, ))
    t2 = threading.Thread(target=thread2_task, args=(lock, 1, ))
 
    t1.start()
    t2.start()
 
    t1.join()
    t2.join()

In [3]:
for i in range(10):
    main_task()
    print("Iteration {0}: x = {1}".format(i,x))

Iteration 0: x = 20000
Iteration 1: x = 20000
Iteration 2: x = 20000
Iteration 3: x = 20000


In [2]:
from peterson import Peterson as p

def increment():
    global x
    x += 1
 
def thread1_task(lock, my_num):
    global turn
   
    for _ in range(100000):

        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def thread2_task(lock, my_num):
    global turn
   
    for _ in range(100000):

        lock.lock(my_num)
        increment()
        lock.unlock(my_num)
 
def main_task():
    global x
 
    x = 0

    lock = p()
   
    t1 = threading.Thread(target=thread1_task, args=(lock, 0, ))
    t2 = threading.Thread(target=thread2_task, args=(lock, 1, ))
 
    t1.start()
    t2.start()
 
    t1.join()
    t2.join()

In [3]:
for i in range(10):
    main_task()
    print("Iteration {0}: x = {1}".format(i,x))

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
      File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
self.run()    self._target(*self._args, **self._kwargs)
  File "/var/folders/mw/tkjdv8_56kd9c3js_hzvflqm0000gp/T/ipykernel_17332/2252348599.py", line 21, in thread2_task
  File "/Users/schoolben/Documents/S22/CS337/CS337-Project-6/peterson.py", line 11, in lock

  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    flag[Thread_ID] = True
NameError: name 'flag' is not defined
    self._target(*self._args, **self._kwargs)
  File "/var/folders/mw/tkjdv8_56kd

Iteration 0: x = 0
Iteration 1: x = 0
Iteration 2: x = 0
Iteration 3: x = 0
Iteration 4: x = 0
Iteration 5: x = 0
Iteration 6: x = 0
Iteration 7: x = 0
Iteration 8: x = 0
Iteration 9: x = 0
